# yo-yo-maskr

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

"LangChain is a popular open-source framework for building blockchain-based applications, particularly focused on the Ethereum ecosystem.\n\nTo understand what LangChain does, let's break it down:\n\n1. **Blockchain platform**: LangChain is built on top of Web3.js and utilizes the Ethereum blockchain as its foundation. This allows developers to create smart contracts and interact with blockchain data.\n2. **Decentralized data storage**: LangChain provides a decentralized data storage solution using IPFS (InterPlanetary File System). This enables seamless data sharing, collaboration, and version control across multiple nodes on the network.\n3. **Smart contract development**: The framework offers a set of tools and libraries for developing smart contracts, making it easier to create, deploy, and manage blockchain-based applications.\n4. **Decentralized application (dApp) building**: LangChain is designed to help developers build dApps that can interact with the blockchain, enabling feat

In [2]:
with open('sample_text.txt', 'r') as file:
    sample_text = file.read()
sample_text

'Hier ist ein kurzer Text mit vielen Namen von Personen und Orten:\n\n---\n\nEs war ein kühler Herbstmorgen, als Marie Müller mit ihrem Bruder Thomas in Hamburg auf den Zug nach Berlin wartete. Dort wollten sie ihre Freunde Anna Schmidt und Lukas Hoffmann treffen, die gerade aus München zurückgekehrt waren. Während der Reise erzählte Marie von ihrem letzten Urlaub in Italien, wo sie in Rom einen alten Schulfreund namens Peter Lehmann getroffen hatte. Peter wohnte mittlerweile in der Nähe von Mailand und arbeitete für eine Firma in Venedig.\n\nIn Berlin angekommen, holte sie ihr Onkel Karl Weber vom Bahnhof ab. Gemeinsam fuhren sie zu seiner Wohnung in der Friedrichstraße, wo sie später an einem Abendessen bei Familie Becker in Charlottenburg teilnehmen wollten. Dort kamen auch Karla Fischer, eine ehemalige Klassenkameradin, und Paul König, ein alter Freund aus der Schulzeit, hinzu.\n\nAm nächsten Tag machten sich Marie und Thomas auf den Weg nach Potsdam, um dort den berühmten Park San